In [1]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets
import random
import torch.nn as nn

SEED = 1234
MAX_VOCAB_SIZE = 25_000
BATCH_SIZE = 64

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data.examples[0]))

train_data, valid_data = train_data.split(random_state = random.seed(SEED))
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")
print(TEXT.vocab.freqs.most_common(20))
print(LABEL.vocab.stoi)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

/Users/zqz_buaa/opt/anaconda3/lib/python3.8/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [26:48<00:00, 52.3kB/s]  


Number of training examples: 25000
Number of testing examples: 25000
{'text': ['For', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem', '.', 'Imagine', 'a', 'movie', 'where', 'Joe', 'Piscopo', 'is', 'actually', 'funny', '!', 'Maureen', 'Stapleton', 'is', 'a', 'scene', 'stealer', '.', 'The', 'Moroni', 'character', 'is', 'an', 'absolute', 'scream', '.', 'Watch', 'for', 'Alan', '"', 'The', 'Skipper', '"', 'Hale', 'jr', '.', 'as', 'a', 'police', 'Sgt', '.'], 'label': 'pos'}
Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


.vector_cache/glove.6B.zip: 862MB [42:12, 340kB/s]                                
100%|█████████▉| 399999/400000 [00:10<00:00, 39420.94it/s]


Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2
[('the', 202478), (',', 192130), ('.', 165401), ('a', 109230), ('and', 109175), ('of', 101087), ('to', 93504), ('is', 76398), ('in', 61293), ('I', 54008), ('it', 53329), ('that', 48904), ('"', 44045), ("'s", 43248), ('this', 42372), ('-', 37498), ('/><br', 35684), ('was', 34978), ('as', 30125), ('with', 29740)]
defaultdict(None, {'neg': 0, 'pos': 1})


In [34]:
class Net(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)        
        self.fc_0 = nn.Linear(embedding_dim, hidden_dim)        
        self.fc_1 = nn.Linear(hidden_dim, output_dim)
        self.act = nn.ReLU()
    def forward(self, text):

        #text = [sent len, batch size]
        embedded = self.embedding(text)
        len_sen = embedded.size()[0]
        embedded = self.act(embedded)
        embedded=torch.sum(embedded,dim=0)/ len_sen
        #embedded = [sent len, batch size, emb dim]
        
        output = self.fc_0(embedded)
        output = self.act(output)
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        output = self.fc_1(output)
        
        return output

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 16
OUTPUT_DIM = 2

model = Net(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
print(f'The model has {count_parameters(model):,} trainable parameters')
print(INPUT_DIM)
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
print(model.embedding.weight.data)

The model has 2,501,850 trainable parameters
25002
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.6018, -0.6654,  0.4103,  ...,  0.1696,  0.6404, -0.2471],
        [ 0.3342,  0.8346, -0.2027,  ...,  0.4676,  0.7014,  0.3065],
        [-0.3864, -0.3255,  0.6744,  ..., -0.2923, -0.3254,  0.7818]])


In [35]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)


def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
  #  rounded_preds = torch.round(torch.sigmoid(preds))
    rounded_preds = torch.max(preds, 1)[1]
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [36]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        if batch.text.size()[1] != batch.label.size()[0]:
            print(batch.text.size()) 
            print(batch.label.size())   
        optimizer.zero_grad()
        # print(batch.text.size()) 
        # print(batch.label.size())
        predictions = model(batch.text).squeeze(1)
 #       predictions = torch.sum(predictions,dim=0) / predictions.size()[0]
        loss = criterion(predictions, batch.label.long())
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            # predictions = torch.sum(predictions,dim=0) / predictions.size()[0]
            loss = criterion(predictions, batch.label.long())
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

import time
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 5

best_valid_loss = float('inf')
print(model)
for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Net(
  (embedding): Embedding(25002, 100)
  (fc_0): Linear(in_features=100, out_features=16, bias=True)
  (fc_1): Linear(in_features=16, out_features=2, bias=True)
  (act): ReLU()
)
Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.689 | Train Acc: 55.19%
	 Val. Loss: 0.641 |  Val. Acc: 66.87%
Epoch: 02 | Epoch Time: 0m 15s
	Train Loss: 0.622 | Train Acc: 72.32%
	 Val. Loss: 0.523 |  Val. Acc: 76.72%
Epoch: 03 | Epoch Time: 0m 15s
	Train Loss: 0.481 | Train Acc: 81.53%
	 Val. Loss: 0.496 |  Val. Acc: 83.09%
Epoch: 04 | Epoch Time: 0m 15s
	Train Loss: 0.376 | Train Acc: 86.58%
	 Val. Loss: 0.477 |  Val. Acc: 86.06%
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.310 | Train Acc: 89.39%
	 Val. Loss: 0.491 |  Val. Acc: 87.46%
